# Graded Challenge 3
# Hasbi Thaufik Oktodila

Untuk merumuskan tindakan dan perencanaan perusahaan, metric SMART dibutuhkan untuk membantu langkah-langkah yang akan diambil

- S-pecific: Meningkatkan keuntungan perusahaan sekitar 5% di tahun ini.

- M-easureable: Meningkatkan sekitar 10-15% penjualan produk perusahaan paling populer di kuartal keempat (Q4).

- A-chievable: Waktu 3 bulan (Q4) merupakan target yang rasional untuk meningkatkan penjualan produk populer perusahaan agar dapat meningkatkan total pendapatan di akhir tahun nanti.

- R-elevant: Meningkatkan penjualan produk paling populer perusahaan merupakan usaha yang relevan untuk meningkatkan tujuan utama yaitu meningkatkan keuntungan di akhir tahun.

- T-ime-bound: Pelaksanaan rencana ini dilakukan di kuartal ke-4 (Q4).


Importing libraries

In [15]:
from google.cloud import bigquery
from google.colab import auth

auth.authenticate_user()
print('Authenticated')

project_id = "hacktiv8-hasbi" #GUNAKAN GCP PROJECT-ID KALIAN MASING-MASING
client = bigquery.Client(project=project_id)

Authenticated


No. 1 : Berapa jumlah transaksi yang berstatus Complete tiap bulan selama Q1 sampai Q3 di tahun 2022? Insight apa yang bisa kamu berikan?

Sebelum menentukan produk apa yang akan difokuskan, kita memerlukan gambaran transaksi yang terjadi di perusahaan selama Q1 - Q3. Hal ini dijelaskan di beberapa tabel di bawah.

In [16]:
df1 = client.query('''
SELECT
  EXTRACT (month FROM created_at) AS month,
  count(status) AS total_completed
FROM (
      SELECT *
      FROM `bigquery-public-data.thelook_ecommerce.orders`
      WHERE status = 'Complete' AND
            created_at BETWEEN '2022-01-01 00:00:00' AND '2022-10-1 00:00:00')
GROUP BY month
ORDER BY month
''').to_dataframe()

In [17]:
df1

,month,total_completed
0,1,1087
1,2,1007
2,3,1303
3,4,1387
4,5,1478
5,6,1543
6,7,1898
7,8,2177
8,9,2443


Dari tabel di atas terlihat Selama Q1 - Q3, total transaksi mengalami kenaikan sampai di akhir Q3. Hanya terjadi penurunan kecil di Februari, tetapi selebihnya, selama 3 kuartal, total transaksi mengalami kenaikan yang signifikan. Tujuan untuk meningkatkan total transaksi di akhir kuartal 4 adalah tujuan yang signifikan.

*Berikan informasi total penjualan (dalam USD) tiap bulan selama Q1 sampai Q3 di tahun 2022! (Hanya yang transaksi berstatus Complete. Apa informasi yang bisa kamu sampaikan?*

In [18]:
df2_1 = client.query('''
SELECT SUM(orders_item.sale_price*orders.num_of_item) AS penjualan_USD,
            EXTRACT(year FROM orders.created_at) AS year,
            EXTRACT(month FROM orders.created_at) AS month,
     FROM bigquery-public-data.thelook_ecommerce.orders AS orders
     INNER JOIN bigquery-public-data.thelook_ecommerce.order_items AS orders_item
     ON orders.order_id = orders_item.order_id
     WHERE orders.status ='Complete' AND 
           orders.created_at >= '2022-01-01' AND 
           orders.created_at<='2022-09-30'
     GROUP BY year,month
     ORDER BY year,month ASC
''').to_dataframe()

df2_1

,penjualan_USD,year,month
0,168952.960231,2022,1
1,162139.870076,2022,2
2,216280.330020,2022,3
3,224725.330124,2022,4
4,234021.129939,2022,5
5,249672.600204,2022,6
6,308199.850179,2022,7
7,355579.690396,2022,8
8,392622.210050,2022,9


In [19]:
Q1 = 168952.960231 + 162139.870076 +	216280.330020
Q2 = 224725.330124 + 	234021.129939 + 249672.600204
Q3 = 308199.850179 + 355579.690396	+ 392622.210050
print(Q1)
print(Q2)
print(Q3)

547373.160327
708419.060267
1056401.7506249999


Sama seperti total transaksi, total pendapatan juga mengalami kenaikan yang signifikan per bulannya. Meningkatkan total pendapatan di kuartal 4 adalah juga relevan.

*No. 3 Berapa user yang melakukan transaksi berstatus Complete di tiap bulan dari Q1 sampai Q3 2022? Apa kesimpulanmu?*

In [20]:
df3_1 = client.query('''
SELECT COUNT(DISTINCT (user_id)) AS total_user,
             EXTRACT(year FROM created_at) AS year,
             EXTRACT(month FROM created_at) AS month,
      
      FROM `bigquery-public-data.thelook_ecommerce.orders`
      WHERE status='Complete' AND 
            created_at >= '2022-01-01' AND 
            created_at<='2022-09-30'
      GROUP BY year, month
      ORDER BY year, month
''').to_dataframe()

df3_1

,total_user,year,month
0,1076,2022,1
1,1000,2022,2
2,1288,2022,3
3,1377,2022,4
4,1463,2022,5
5,1518,2022,6
6,1874,2022,7
7,2148,2022,8
8,2282,2022,9


Dari tabel di atas, terlihat juga jumlah user yang mengalami kenaikan dalam hal jumlah transaksi. Berdasarkan ketiga tabel di atas, user, total pendapatan, dan jumlah transaksi mengalami kenaikan selama kuartal 1-3.

Sekarang, kita masuk ke produk paling populer di perusahaan agar dapat makin ditingkatkan penjualannya untuk menarik lebih banyak pendapatan.

*No. 4 Kategori produk apa saja yang paling banyak dibeli (staus transaksi: Complete) di tiap bulannya selama Q1 sampai Q3 tahun 2022? Berikan insight!*

In [21]:
df4 = client.query('''
SELECT table_2.category, 
       table_2.total_transaksi, 
       table_2.month
FROM
      (SELECT table_1.*,
              ROW_NUMBER() OVER(PARTITION BY month ORDER BY month ASC, total_transaksi DESC) AS baris 
       FROM
          (SELECT ii.product_category AS category, 
                  COUNT(*) AS total_transaksi,
                  EXTRACT (month FROM oi.delivered_at) AS month
          FROM `bigquery-public-data.thelook_ecommerce.order_items` AS oi 
          JOIN `bigquery-public-data.thelook_ecommerce.inventory_items` AS ii
          ON oi.product_id = ii.product_id
          WHERE oi.status = 'Complete' AND
                oi.delivered_at >= '2022-01-01' AND
                oi.delivered_at <= '2022-09-30'
                GROUP BY category, month
                ORDER BY month ASC, total_transaksi DESC) AS table_1) AS table_2
                WHERE table_2.baris = 1
ORDER BY table_2.month
''').to_dataframe()

df4

,category,total_transaksi,month
0,Intimates,2320,1
1,Intimates,2303,2
2,Jeans,2691,3
3,Intimates,2712,4
4,Jeans,3072,5
5,Intimates,3318,6
6,Jeans,3956,7
7,Tops & Tees,4360,8
8,Intimates,4686,9


Dari data di atas, terlihat bahwa **Intimates**, **Jeans**, dan **Tops & Tees**, merupakan kategori paling banyak meng-generate jumlah transaksi selama Q1 - Q3. Kita sekarang sudah mendapatkan gambaran mengenai kategori apa yang harus ditingkatkan.

*No. 5: Kategori produk apa saja yang paling banyak mendapatkan pendapatan (staus transaksi: Complete) di tiap bulannya selama Q1 sampai Q3 tahun 2022? Berikan insight!*

In [22]:
df5 = client.query('''
SELECT table_2.category,
       table_2.total_pendapatan,
       table_2.month
FROM (SELECT table_1.*,
             ROW_NUMBER() OVER(PARTITION BY month ORDER BY month ASC, total_pendapatan DESC) as baris
      FROM (SELECT ii.product_category AS category, 
                   SUM(oi.sale_price) AS total_pendapatan,
            EXTRACT (month FROM oi.delivered_at) AS month
            FROM `bigquery-public-data.thelook_ecommerce.order_items` AS oi 
            JOIN `bigquery-public-data.thelook_ecommerce.inventory_items` as ii
            ON oi.product_id = ii.product_id
            WHERE oi.status = 'Complete' AND
                  oi.delivered_at >= '2022-01-01' AND
                  oi.delivered_at <= '2022-09-30'
            GROUP BY ii.product_category, month
            ORDER BY month ASC) AS table_1) AS table_2
WHERE table_2.baris = 1
ORDER BY table_2.month
''').to_dataframe()

df5

,category,total_pendapatan,month
0,Jeans,202457.830278,1
1,Outerwear & Coats,253020.719183,2
2,Jeans,272914.940231,3
3,Outerwear & Coats,294076.730497,4
4,Jeans,329144.910536,5
5,Outerwear & Coats,314451.289055,6
6,Outerwear & Coats,503568.659752,7
7,Outerwear & Coats,447885.089411,8
8,Outerwear & Coats,498404.238115,9


Lalu, dari tabel di atas, **kategori Jeans** dan **Outwear & Coats** merupakan kategori yang paling menggenerate pendapatan.

No. 6 Dibandingkan dengan jumlah transaksi dan total penjualan, mana yang paling berkaitan dengan jumlah user yang melakukan transaksi? Apa analisis yang dapat kamu jelaskan?

Lalu, bagaimana dengan korelas antar **jumlah user <-> jumlah transaksi**, dan **jumlah user <-> total penjualan**?

In [23]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [24]:
df6 = client.query('''
SELECT --table_2.category,
       table_2.total_pendapatan,
       --table_2.bulan,
       table_2.total_complete,
       table_2.jumlah_user
FROM (SELECT table_1.*,
             ROW_NUMBER() OVER(PARTITION BY bulan ORDER BY bulan ASC, total_pendapatan DESC) as baris
      FROM (SELECT ii.product_category AS category, 
                   SUM(oi.sale_price) AS total_pendapatan,
                   COUNT(oi.status) AS total_complete,
                   COUNT (DISTINCT(oi.user_id)) AS jumlah_user,
                   EXTRACT(month FROM oi.delivered_at) AS bulan
            FROM `bigquery-public-data.thelook_ecommerce.order_items` AS oi 
            JOIN `bigquery-public-data.thelook_ecommerce.inventory_items` as ii
            ON oi.product_id = ii.product_id
            WHERE oi.status = 'Complete' AND
                  oi.delivered_at >= '2022-01-01' AND
                  oi.delivered_at <= '2022-09-30'
            GROUP BY ii.product_category, bulan
            ORDER BY bulan ASC) AS table_1) AS table_2
WHERE table_2.baris = 1
ORDER BY table_2.bulan
''').to_dataframe()

df6

,total_pendapatan,total_complete,jumlah_user
0,202457.830278,2112,100
1,253020.719183,1541,74
2,272914.940231,2691,135
3,294076.730497,2279,112
4,329144.910536,3072,139
5,314451.289055,2195,108
6,503568.659752,2813,124
7,447885.089411,3270,161
8,498404.238115,3421,171


In [25]:
df6.corr()

,total_pendapatan,total_complete,jumlah_user
total_pendapatan,1.000000,0.757314,0.713097
total_complete,0.757314,1.000000,0.979186
jumlah_user,0.713097,0.979186,1.000000


Dari angka di atas, korelasi **user -- pendapatan** mendapatkan nilai 0.7 sedangkan **user -- total transaksi** mendapatkan nilai 0.9. Hal ini mengindikasikan bahwa jumlah user lebih berkorelasi terhadap total transaksi ketimbang jumlah pendapatan. Hal ini bisa jadi disebabkan jumlah pendapatan lebih ditentukan oleh jumlah barang yang dibeli, karena banyak user yang membeli banyak barang dalam satu transaksi. Sementara jika dibandingkan dengan jumlah transaksi, banyaknya user sangat berkorelasi sehingga menggenerate total transaksi yang terjadi. 

#Poin Analisis

Apakah problem statement yang kamu definisikan di awal dapat terukur ketercapaiannya? berikan pendapatmu!

- Bisa. Untuk mencapai peningkatan keuntungan perusahaan sebanyak 10% dari tahun sebelumnya dapat diukur kesuksesannya dengan membandingkan pendapatan tahun ini dengan tahun sebelumnya.

Berdasarkan hasil analisis yang sudah kamu lakukan dari 6 penjabaran di atas, jika CEO perusahaanmu ingin menargetkan pendapatan di awal kuartal 4 harus mencapai $250000, apakah masuk akal?

- Masuk akal. Dari tabel no.2 dapat dihitung bahwa pendapatan di awal quartal selalu melebihi angka 250,000. Tujuan tersebut masih rasional, bahkan bisa menargetkan target lebih.

CEO kamu menargetkan di kuartal 4 ada investor yang dapat menyuntikan dana ke perusahaanmu, dimana investor akan melihat GMV selama 3 kuartal terakhir serta prospek kedepan minimal di kuartal 4 akan seperti apa. Berikan informasi kepada CEO mu berdasarkan trend transaksi, jumlah user yang bertransaksi, dan GMV 3 kuartal terakhir, apakah perusahaanmu layak atau tidak mendapatkan investor baru!

- Salah satu hal yang terpenting bagi investor adalah stabilitas. Jika dilihat dari jumlah pendapatan dan transaksi, The Look sudah memiliki data kenaikan yang signifikan selama tiga kuartal ini. Poin kedua, dari segi popularitas produk. Dapat dilihat dari tabel ke-5, kategori produk **Jeans** dan **Outwear and Coats** selalu memberikan pendapatan yang tinggi. Ini menunjukkan ada stabilitas dan loyalitas pasar terhadap salah satu kategori produk. Stabilitas ini merupakan salah satu faktor yang dapat digali lebih mendalam dan dapat diperlihatkan kepada investor mengenai stabilitas produk kita.





# Kesimpulan

Untuk meningkatkan pendapatan perusahaan, salah satu langkah yaitu melakukan improvement dan menjaga stabilitas produk paling populer perusahaan. Kategori produk tersebut adalah **Jeans** dan **Outwear and Coats** Hal ini dapat menjaga kesehatan keuangan perusahaan sambil menjadi pijakan aman jika ingin melakukan terobosan-terobosan baru.